In [ ]:
import requests
from bs4 import BeautifulSoup

# Créez des listes pour stocker les données
Noms = []
Adresses = []
Numeros = []
Secteurs = []
Liens = []

# Liste des pays avec leurs codes
pays = [
    "za", "gn", "cm", "dz", "ao", "bj", "bw", "bf", "cm", "cg", "cd", "ci", "dj", "eg", "et", "ga", "gh", 
    "gn", "ke", "lr", "mg", "mw", "ml", "ma", "mu", "mz", "na", "ne", "ng", "ug", "cf", "rw", "sn", "so", 
    "tz", "tg", "tn"
]

# Boucle à travers les pages pour chaque pays
for code in pays:
    for page_num in range(1, 749):  # Changer le nombre de pages selon vos besoins
        # URL de la page à récupérer avec le numéro de page variable
        url = f"https://www.goafricaonline.com/{code}/annuaire-resultat?type=company&p={page_num}"
        # Envoie une requête GET à l'URL
        response = requests.get(url)
        # Vérifie si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Utilise BeautifulSoup pour analyser le contenu HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trouver toutes les balises <a> avec la classe spécifiée pour les numéros de téléphone
            numeros = soup.find_all('a', class_='z-10 text-13 t:text-14 text-gray-700 underline hover:no-underline')

            # Trouver toutes les balises <a> avec la classe spécifiée pour les noms
            noms = soup.find_all('a', class_='stretched-link font-bold text-16 t:text-20 text-black hover:text-black no-underline hover:no-underline')

            # Trouver toutes les balises <address> avec la classe spécifiée pour les adresses
            adresses = soup.find_all('address', class_='text-14 text-gray-700 flex-1')

            # Trouver toutes les balises <div> avec la classe spécifiée pour les secteurs d'activité
            secteurs = soup.find_all('div', class_='text-14 text-brand-blue mb-4')

            # Trouver toutes les balises <div> pour les liens des sites web
            liens = soup.find_all('div', class_='flex items-center flex-col mm:flex-row t:px-8 z-10 group cursor-pointer no-underline hover:no-underline')

            # Extraire les liens de cette page
            links_on_page = []
            for lien in liens:
                # Extraire la valeur de l'attribut 'data-collect-event-on-click'
                data_collect_event = lien['data-collect-event-on-click']

                # Convertir la chaîne JSON en un dictionnaire Python
                data_collect_event_dict = json.loads(data_collect_event)

                # Extraire la valeur de la clé "link"
                link_value = data_collect_event_dict.get('link', None)
                links_on_page.append(link_value)

            # Vérifier si des numéros de téléphone sont présents
            if numeros:
                for i, element in enumerate(numeros):
                    # Extraire le numéro de téléphone
                    numero_text = element.get_text(strip=True)

                    # Extraire le nom correspondant
                    if i < len(noms):
                        nom_text = noms[i].get_text(strip=True)
                    else:
                        nom_text = None  # Si le nom n'est pas présent, ajouter None

                    # Extraire l'adresse correspondante
                    if i < len(adresses):
                        adresse_text = adresses[i].get_text(strip=True)
                    else:
                        adresse_text = None  # Si l'adresse n'est pas présente, ajouter None

                    # Extraire le secteur d'activité correspondant
                    if i < len(secteurs):
                        secteur_text = secteurs[i].get_text(strip=True)
                    else:
                        secteur_text = None  # Si le secteur d'activité n'est pas présent, ajouter None

                    # Extraire le lien correspondant de la liste des liens de cette page
                    if i < len(links_on_page):
                        link_value = links_on_page[i]
                    else:
                        link_value = None  # Si le lien n'est pas présent, ajouter None

                    # Ajouter les données à leurs listes respectives
                    Noms.append(nom_text)
                    Adresses.append(adresse_text)
                    Numeros.append(numero_text)
                    Secteurs.append(secteur_text)
                    Liens.append(link_value)

        else:
            print(f"La requête pour la page {page_num} a échoué avec le code de statut :", response.status_code)

    df = pd.DataFrame({'Noms': Noms, 'Adresses': Adresses, 'Numeros': Numeros, 'Secteurs': Secteurs})        


    liste_sans_doublons = list(set(Liens))


    # Créez une liste de liens à parcourir
    liste_de_liens = liste_sans_doublons

    def extract_emails_from_soup(soup):
        emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', soup.text)
        return emails

    # Créez une liste pour stocker les informations récupérées
    informations = []

    for lien in liste_de_liens:
        try:
            # Envoie une requête GET à l'URL
            response = requests.get(lien)

            # Vérifiez si la requête a réussi (statut 200)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Utilisez les fonctions extract_emails_from_soup et extract_phone_numbers_from_soup pour extraire les informations
                emails = extract_emails_from_soup(soup)

                info_site = {'Liens': lien}

                # Si des adresses email sont trouvées, prenez la première
                if emails:
                    info_site['Email (fonction)'] = emails[0]
                else:
                    info_site['Email (fonction)'] = None

                # Ajoutez les informations du site actuel à la liste
                informations.append(info_site)
            else:
                print(f"La requête pour le lien {lien} a échoué avec le code de statut :", response.status_code)
        except Exception as e:
            print(f"Une erreur s'est produite lors de la recherche sur le lien {lien}: {str(e)}")

    # Créez un DataFrame à partir des informations récupérées
    df1 = pd.DataFrame(informations)

    # Fonction pour nettoyer un texte en supprimant les espaces et les apostrophes
    def nettoyer_texte(texte):
        texte = texte.lower()  # Convertir le texte en minuscules
        texte = texte.replace("'", "")  # Supprimer les apostrophes
        return texte

    # Fonction pour trouver le mot le plus long dans une chaîne de texte
    def mot_plus_long(texte):
        mots = texte.split()
        return max(mots, key=len)

    # Fonction pour vérifier si un lien correspond à un nom en utilisant le mot le plus long dans le nom
    def verifie_correspondance(nom, lien):
        if lien is not None:  # Vérifiez si le lien n'est pas None
            nom_nettoye = nettoyer_texte(nom)
            lien_nettoye = nettoyer_texte(lien)

            # Trouver le mot le plus long dans le nom
            mot_plus_long_nom = mot_plus_long(nom_nettoye)

            # Comparer la succession des lettres du mot le plus long dans le nom avec le lien
            return mot_plus_long_nom in lien_nettoye
        else:
            return False  # Si le lien est None, il ne peut pas correspondre à un nom



    # Créer une liste vide pour stocker les correspondances
    correspondances_trouvees = []


    # Parcourir chaque nom et chaque lien
    for nom in Noms:
        correspondance_trouvee = False  # Pour vérifier si une correspondance a été trouvée pour ce nom
        for lien in Liens:
            correspondance = verifie_correspondance(nom, lien)
            if correspondance:
                correspondances_trouvees.append({'Noms': nom, 'Liens': lien})
                correspondance_trouvee = True  # Indiquer qu'une correspondance a été trouvée pour ce nom
        if not correspondance_trouvee:
            correspondances_trouvees.append({'Noms': nom, 'Liens': None})  # Aucune correspondance trouvée pour ce nom


    # Créer un dataframe à partir de la liste de correspondances
    df2 = pd.DataFrame(correspondances_trouvees)

    resultat = pd.merge(df2, df1, on="Liens", how="left")

    final = pd.merge(df, resultat[['Noms', 'Email (fonction)']], left_on='Noms', right_on='Noms', how='left')
    final = final.drop_duplicates(subset='Noms', keep='first')

    nombre_de_valeurs_non_nulles = final['Email (fonction)'].count()

    # Afficher le nombre de valeurs non nulles
    print("Nombre de valeurs non nulles dans la colonne 'Emails':", nombre_de_valeurs_non_nulles)

    final

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Créez des listes pour stocker les données
Noms = []
Adresses = []
Numeros = []
Secteurs = []
Liens = []
Pays = []

# Liste des pays avec leurs codes
pays = [
    "za", "gn", "cm", "dz", "ao", "bj", "bw", "bf", "cm", "cg", "cd", "ci", "dj", "eg", "et", "ga", "gh",
    "gn", "ke", "lr", "mg", "mw", "ml", "ma", "mu", "mz", "na", "ne", "ng", "ug", "cf", "rw", "sn", "so",
    "tz", "tg", "tn"
]

# Boucle à travers les pages pour chaque pays
for code in pays:
    for page_num in range(1, 749):  # Changer le nombre de pages selon vos besoins
        # URL de la page à récupérer avec le numéro de page variable
        url = f"https://www.goafricaonline.com/{code}/annuaire-resultat?type=company&p={page_num}"
        # Envoie une requête GET à l'URL
        response = requests.get(url)
        # Vérifie si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Utilise BeautifulSoup pour analyser le contenu HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trouver toutes les balises <a> avec la classe spécifiée pour les numéros de téléphone
            numeros = soup.find_all('a', class_='z-10 text-13 t:text-14 text-gray-700 underline hover:no-underline')

            # Trouver toutes les balises <a> avec la classe spécifiée pour les noms
            noms = soup.find_all('a', class_='stretched-link font-bold text-16 t:text-20 text-black hover:text-black no-underline hover:no-underline')

            # Trouver toutes les balises <address> avec la classe spécifiée pour les adresses
            adresses = soup.find_all('address', class_='text-14 text-gray-700 flex-1')

            # Trouver toutes les balises <div> avec la classe spécifiée pour les secteurs d'activité
            secteurs = soup.find_all('div', class_='text-14 text-brand-blue mb-4')

            # Vérifier si des numéros de téléphone sont présents
            if numeros:
                for i, element in enumerate(numeros):
                    # Extraire le numéro de téléphone
                    numero_text = element.get_text(strip=True)

                    # Extraire le nom correspondant
                    if i < len(noms):
                        nom_text = noms[i].get_text(strip=True)
                    else:
                        nom_text = None  # Si le nom n'est pas présent, ajouter None

                    # Extraire l'adresse correspondante
                    if i < len(adresses):
                        adresse_text = adresses[i].get_text(strip=True)
                    else:
                        adresse_text = None  # Si l'adresse n'est pas présente, ajouter None

                    # Extraire le secteur d'activité correspondant
                    if i < len(secteurs):
                        secteur_text = secteurs[i].get_text(strip=True)
                    else:
                        secteur_text = None  # Si le secteur d'activité n'est pas présent, ajouter None
                    # Ajouter les données à leurs listes respectives
                    Noms.append(nom_text)
                    Adresses.append(adresse_text)
                    Numeros.append(numero_text)
                    Secteurs.append(secteur_text)
                    Pays.append(code)

        else:
            print(f"La requête pour la page {page_num} a échoué avec le code de statut :", response.status_code)

In [3]:
# Créer un DataFrame pour les données initiales
df = pd.DataFrame({'Noms': Noms, 'Adresses': Adresses, 'Numeros': Numeros, 'Secteurs': Secteurs, 'Pays': Pays})
df

,Noms,Adresses,Numeros,Secteurs,Pays
0,0861 TRAVEL,18 Kerk StBethlehem - Afrique du Sud,Tel:(+27)05 83 03 73 97,Agences de voyage,za
1,10 2ND AVENUE HOUGHTON ESTATE,"10, 2ND Avenue, Houghton Estate, Johannesburg,...",Tel:(+27)01 18 53 24 00,Hôtels,za
2,10 CADOZA BED AND BREAKFAST IN WESTDENE,"10 Cadoza street Westdene, Johannesburg, 2092 ...",Tel:(+27)116 73 30 03,Hôtels,za
3,130 MARKET BED AND BREAKFAST,"130 Market Street, Johannesburg North, Johanne...",Tel:(+27)837 41 68 88,Hôtels,za
4,15 ON ORANGE GUEST HOUSE,"15 Orange Road, Johannesburg, Gauteng, 2192, S...",Tel:(+27)08 34 49 11 40,Hôtels,za
...,...,...,...,...,...
264,ASCENSION PROP LTD A,"3e étage, Palazzo Towers Ouest, Montecasino Bo...",Tel:(+27)01 15 11 53 35,Immobilier,za
265,ASH MANOR LUXURY GUESTHOUSE,"17A Ashley Avenue, Bryanston, South Africa, 20...",Tel:(+27)01 17 06 19 00,Hôtels,za
266,ASH TRAVEL CC,"23 Antrim Rd, Three Anchor Bay, Sea PointSea P...",Tel:(+27)02 14 33 15 58,Agences de voyage,za
267,ASHANTI TRAVEL ONLINE,11 Hof Street 8001 Le CapCape Town - Afrique d...,Tel:(+27)212 02 34 71,Agences de voyage,za


In [ ]:
# Supprimer les doublons dans la colonne Liens
df = df.drop_duplicates(subset='Liens')

# Créer une liste de liens à parcourir
liste_de_liens = df['Liens'].tolist()

# Créer une liste pour stocker les informations récupérées
informations = []

def extract_emails_from_soup(soup):
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', soup.text)
    return emails

# Parcourir chaque lien
for lien in liste_de_liens:
    try:
        # Envoie une requête GET à l'URL
        response = requests.get(lien)

        # Vérifiez si la requête a réussi (statut 200)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Utilisez les fonctions extract_emails_from_soup et extract_phone_numbers_from_soup pour extraire les informations
            emails = extract_emails_from_soup(soup)

            info_site = {'Liens': lien}

            # Si des adresses email sont trouvées, prenez la première
            if emails:
                info_site['Email (fonction)'] = emails[0]
            else:
                info_site['Email (fonction)'] = None

            # Ajoutez les informations du site actuel à la liste
            informations.append(info_site)
        else:
            print(f"La requête pour le lien {lien} a échoué avec le code de statut :", response.status_code)
    except Exception as e:
        print(f"Une erreur s'est produite lors de la recherche sur le lien {lien}: {str(e)}")

# Créer un DataFrame à partir des informations récupérées
df1 = pd.DataFrame(informations)

# Fonction pour nettoyer un texte en supprimant les espaces et les apostrophes
def nettoyer_texte(texte):
    texte = texte.lower()  # Convertir le texte en minuscules
    texte = texte.replace("'", "")  # Supprimer les apostrophes
    return texte

# Fonction pour trouver le mot le plus long dans une chaîne de texte
def mot_plus_long(texte):
    mots = texte.split()
    return max(mots, key=len)

# Fonction pour vérifier si un lien correspond à un nom en utilisant le mot le plus long dans le nom
def verifie_correspondance(nom, lien):
    if lien is not None:  # Vérifiez si le lien n'est pas None
        nom_nettoye = nettoyer_texte(nom)
        lien_nettoye = nettoyer_texte(lien)

        # Trouver le mot le plus long dans le nom
        mot_plus_long_nom = mot_plus_long(nom_nettoye)

        # Comparer la succession des lettres du mot le plus long dans le nom avec le lien
        return mot_plus_long_nom in lien_nettoye
    else:
        return False  # Si le lien est None, il ne peut pas correspondre à un nom

# Créer une liste vide pour stocker les correspondances
correspondances_trouvees = []

# Parcourir chaque nom et chaque lien
for nom in Noms:
    correspondance_trouvee = False  # Pour vérifier si une correspondance a été trouvée pour ce nom
    for lien in Liens:
        correspondance = verifie_correspondance(nom, lien)
        if correspondance:
            correspondances_trouvees.append({'Noms': nom, 'Liens': lien})
            correspondance_trouvee = True  # Indiquer qu'une correspondance a été trouvée pour ce nom
    if not correspondance_trouvee:
        correspondances_trouvees.append({'Noms': nom, 'Liens': None})  # Aucune correspondance trouvée pour ce nom

# Créer un DataFrame à partir de la liste de correspondances
df2 = pd.DataFrame(correspondances_trouvees)

# Fusionner les DataFrames pour obtenir les informations finales
resultat = pd.merge(df2, df1, on="Liens", how="left")
final = pd.merge(df, resultat[['Noms', 'Email (fonction)']], left_on='Noms', right_on='Noms', how='left')

# Supprimer les doublons dans le DataFrame final
final = final.drop_duplicates(subset='Noms', keep='first')

# Afficher le DataFrame final
print(final.head())

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Créez des listes pour stocker les données
Noms = []
Adresses = []
Numeros = []
Secteurs = []
Liens = []
Pays = []
Villes = []
Sites = []
Tel = []
Numeros = []

# Liste des pays avec leurs codes
pays = [
    "za", "gn", "cm", "dz", "ao", "bj", "bw", "bf", "cm", "cg", "cd", "ci", "dj", "eg", "et", "ga", "gh",
    "gn", "ke", "lr", "mg", "mw", "ml", "ma", "mu", "mz", "na", "ne", "ng", "ug", "cf", "rw", "sn", "so",
    "tz", "tg", "tn"
]

import re

for code in pays:
    for page_num in range(1, 749):  # Adjust the number of pages as needed
        url = f"https://www.goafricaonline.com/{code}/annuaire-resultat?type=company&p={page_num}"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all articles containing company information
            articles = soup.find_all('article', {'data-role': 'company'})

            for article in articles:
                # Extract information from each article
                company_name = article.find('h2').find('a').text.strip()

                # Check if the industry information is available
                industry_element = article.find('div', {'class': 'text-14 text-brand-blue'})
                industry = industry_element.text.strip() if industry_element else ''

                address_element = article.find('address')
                address_text = address_element.text.strip() if address_element else ''

                # Extract the href attribute
                company_link = article.find('h2').find('a')['href']

                phone_number_element = article.find('a', {'href': True, 'data-collect-event-on-click': True})
                phone = phone_number_element.text.strip() if phone_number_element else ''

                # Extract city and country from address_text
                address_parts = address_text.split('-')

                if len(address_parts) > 1:
                    country = address_parts[1].strip()  # Last word before the "-"
                    city_match = re.search(r'[A-Z](?=[^A-Z]*$)', address_parts[0].strip())
                    if city_match:
                        start_index = city_match.start()
                        city = address_parts[0].strip()[start_index:].strip()
                    else:
                        city = ''
                # Additional information (optional)
                description_element = article.find('div', {'class': 'text-gray-700 text-14'})
                description = description_element.text.strip() if description_element else ''
                
                company_response = requests.get(company_link)
                if company_response.status_code == 200:
                    company_soup = BeautifulSoup(company_response.text, 'html.parser')

                    # Extraction du site web
                    website_link_element = company_soup.find('a', class_='font-bold text-link block')

                    if website_link_element:
                        website_link = website_link_element['href']
                    else:
                        website_link = ""

                    phone_numbers_elements = company_soup.find_all('a', class_='text-13 text-gray-700 underline hover:no-underline')

                    if phone_numbers_elements:
                        for phone_number_element in phone_numbers_elements:
                            phone_number = phone_number_element['href']
                            
                    else:
                        phone_number = ""
                else:
                    print(f"Erreur: {company_response.status_code}")
                    

                # Append the extracted information to the lists
                Noms.append(company_name)
                Secteurs.append(industry)
                Adresses.append(address_text)
                Sites.append(website_link)
                Numeros.append(phone_number)
                Tel.append(phone)
                Liens.append(company_link)  # Add the company link
                Villes.append(city)  # Add the city
                Pays.append(country)  # Add the country

KeyboardInterrupt: 

,Nom,Secteur,Sites,Adresse,Numero,Tel,Liens,Ville,Pays
0,0861 TRAVEL,,,18 Kerk StBethlehem - Afrique du Sud,tel:+27 05 83 03 73 97,Tel: (+27) 05 83 03 73 97,https://www.goafricaonline.com/za/124626-0861-...,Bethlehem,Afrique du Sud
1,10 2ND AVENUE HOUGHTON ESTATE,,http://www.houghtonestate.com,"10, 2ND Avenue, Houghton Estate, Johannesburg,...",tel:+27 01 14 83 30 37,Tel: (+27) 01 18 53 24 00,https://www.goafricaonline.com/za/310488-10-2n...,Johannesburg,Afrique du Sud
2,10 CADOZA BED AND BREAKFAST IN WESTDENE,,,"10 Cadoza street Westdene, Johannesburg, 2092 ...",tel:+27 833 60 75 76,Tel: (+27) 116 73 30 03,https://www.goafricaonline.com/za/310489-10-ca...,Johannesburg,Afrique du Sud
3,130 MARKET BED AND BREAKFAST,,,"130 Market Street, Johannesburg North, Johanne...",tel:+27 837 41 68 88,Tel: (+27) 837 41 68 88,https://www.goafricaonline.com/za/310490-130-m...,Johannesburg,Afrique du Sud
4,15 ON ORANGE GUEST HOUSE,,http://www.fifteenonorange.co.za,"15 Orange Road, Johannesburg, Gauteng, 2192, S...",tel:+27 01 17 28 78 70,Tel: (+27) 08 34 49 11 40,https://www.goafricaonline.com/za/310491-15-on...,Johannesburg,Afrique du Sud
5,1TIME HOLDINGS LTD,,,"Unit D2, Isando Business Park, Gewel Street, I...",tel:+27 110 86 81 00,Tel: (+27) 110 86 81 00,https://www.goafricaonline.com/za/310492-1time...,Isando,Afrique du Sud
6,2 LEAFED DOORS,,http://www.2leafeddoors.co.za,"25 Belgrave Road, Sandton, BryanstonJohannesbu...",tel:+27 01 14 63 87 87,Tel: (+27) 01 14 63 87 87,https://www.goafricaonline.com/za/310493-2-lea...,Johannesburg,Afrique du Sud
7,2 WAY TRAVEL,,http://www.2waytravel.com/,"27 Main Road, 7700Cape Town - Afrique du Sud",tel:+27 02 16 86 59 98,Tel: (+27) 02 16 86 59 98,https://www.goafricaonline.com/za/124627-2-way...,Town,Afrique du Sud
8,2004 PROMOTIONS CC,,,"1 Forest Creek Estate, 58 Forest Dr, Forest Hi...",tel:+27 03 17 62 24 21,Tel: (+27) 03 17 62 24 02,https://www.goafricaonline.com/za/124628-2004-...,Durban,Afrique du Sud
9,2010 TOURIST,,,"Jeppe Street, 0001Johannesburg - Afrique du Sud",tel:+27 01 23 43 11 45,Tel: (+27) 01 23 43 11 45,https://www.goafricaonline.com/za/124629-2010-...,Johannesburg,Afrique du Sud


In [45]:
import re

# Exemple d'adresse
address_text = "130 Market Street, Johannesburg North, Johannesburg, Gauteng, 2188 South Africa"

# Trouver la dernière lettre majuscule
city_match = re.search(r'[A-Z](?=[^A-Z]*$)', address_text)

if city_match:
    start_index = city_match.end()
    city = address_text[start_index:].strip()
else:
    city = ''

print("City:", city)

City: frica


In [46]:
import re

part1 = address_parts[0].strip()

# Trouver la dernière lettre majuscule
city_match = re.search(r'[A-Z](?=[^A-Z]*$)', part1)

if city_match:
    start_index = city_match.start()
    city = address_text[start_index:].strip()
else:
    city = ''

print("City:", city)

City: Africa


In [50]:
address_text = "123 Main Street - Cityville"
address_parts = address_text.split('-')

# Afficher les deux parties
part1 = address_parts[0].strip()  # Enlever les espaces autour de la première partie
part2 = address_parts[1].strip()  # Enlever les espaces autour de la deuxième partie

print("Partie 1:", part1)
print("Partie 2:", part2)

Partie 1: 123 Main Street
Partie 2: Cityville
